In [214]:
import pandas as pd
import json

Per la realizzazione ho seguito https://www.datacamp.com/tutorial/text-analytics-beginners-nltk

In [215]:
dataset = pd.read_csv('description_predictions.csv', index_col=0)
with open("sinonimi.json") as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()

In [216]:
dataset_withoutNAN = dataset[dataset.prediction != 'NAN']

dataset_withoutNAN.reset_index(drop=True, inplace=True)

Il codice seguente serve per convertire i soprannomi delle squadre in home team e away team

In [217]:
"""for i, row in dataset_withoutNAN.iterrows():
    h_team, a_team, description, prediction = row.h_team, row.a_team, row.description, row.prediction

    syn = {}
    #cerco nel dizionario di sinonimi, tutti i sinonimi delle squadre del match
    for key in jsonObject.keys():
        if (h_team in key) or (key in h_team):
            syn['home team'] = jsonObject[key] 
            
        if (a_team in key) or (key in a_team):
            syn['away team'] = jsonObject[key] 

    #successivamente prendo il testo e sostituisco i sinonimi con home o away team
    description = description.lower()
    for key in syn.keys():
        for val in syn[key]:
            description = description.replace(val.lower(), key)

    dataset_withoutNAN.at[i, 'description'] = description
"""

"for i, row in dataset_withoutNAN.iterrows():\n    h_team, a_team, description, prediction = row.h_team, row.a_team, row.description, row.prediction\n\n    syn = {}\n    #cerco nel dizionario di sinonimi, tutti i sinonimi delle squadre del match\n    for key in jsonObject.keys():\n        if (h_team in key) or (key in h_team):\n            syn['home team'] = jsonObject[key] \n            \n        if (a_team in key) or (key in a_team):\n            syn['away team'] = jsonObject[key] \n\n    #successivamente prendo il testo e sostituisco i sinonimi con home o away team\n    description = description.lower()\n    for key in syn.keys():\n        for val in syn[key]:\n            description = description.replace(val.lower(), key)\n\n    dataset_withoutNAN.at[i, 'description'] = description\n"

In [218]:
dataset_withoutNAN = dataset_withoutNAN[['description', 'prediction']]
prediction_labels = {
    'N': 0,
    'V': 1,
    'P': 2
}

dataset_withoutNAN['prediction'] = dataset_withoutNAN['prediction'].map(prediction_labels)
dataset_withoutNAN.head()

,description,prediction
0,Verona and Sassuolo take on each other in a hi...,0
1,Inter begin their title defence with a match a...,1
2,Atalanta will look to make a flying start to t...,2
3,Newcomers Empoli will be aiming to kick off th...,2
4,After failing to impress in the 2020/21 Serie ...,2


In [219]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(2,2)) #vectorizer sarà il nostro modello da allenare
X = vectorizer.fit_transform(dataset_withoutNAN.description) #impara il vocabolario e crea Idf, poi ritorna la matrice document-term

print(X.shape)

(379, 11211)


In [221]:
from nltk.corpus import stopwords
import nltk
stop_words = set(stopwords.words('english'))

# this function returns a list of tokenized and stemmed words of any text
def get_tokenized_list(doc_text):
    tokens = nltk.word_tokenize(doc_text)
    return tokens

# This function will performing stemming on tokenized words
def word_stemmer(token_list):
  ps = nltk.stem.PorterStemmer()
  stemmed = []
  for words in token_list:
    stemmed.append(ps.stem(words))
  return stemmed

In [222]:
# Function to remove stopwords from tokenized word list
def remove_stopwords(doc_text):
  cleaned_text = []
  for words in doc_text:
    if words not in stop_words:
      cleaned_text.append(words)
  return cleaned_text

In [223]:
cleaned_corpus = []
for doc in dataset_withoutNAN.description:
  tokens = get_tokenized_list(doc)
  doc_text = remove_stopwords(tokens)
  doc_text  = word_stemmer(doc_text)
  doc_text = ' '.join(doc_text)
  cleaned_corpus.append(doc_text)

In [224]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [225]:

vectorizer = TfidfVectorizer(ngram_range=(2,2)) #vectorizer sarà il nostro modello da allenare
text_counts = vectorizer.fit_transform(cleaned_corpus) #impara il vocabolario e crea Idf, poi ritorna la matrice document-term
df1 = pd.DataFrame(text_counts.toarray(), columns=vectorizer.get_feature_names())

In [226]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    text_counts, dataset_withoutNAN.prediction, test_size=0.3, random_state=1)

In [227]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as mtr


models = {
    'Logistic Regression': LogisticRegression(max_iter=10000), #max_iter di default vale 100, ho dovuto alzarlo se no non converge
    'Support Vector Machine': SVC(),
    'Multinomial Naive Bayes': MultinomialNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'K-Nearest Neighbors': KNeighborsClassifier()
}

In [228]:
predictions = {}
for model_name, model in models.items():
    models[model_name].fit(X_train, y_train)
    predictions[model_name] = model.predict(X_test)

E = []
for estimator, y_pred in predictions.items():
    report = mtr.classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    E.append({
        'Model': estimator, 'Accuracy': report['accuracy'],
        'Avg Precision (macro)': report['macro avg']['precision'],
        'Avg Recall (macro)': report['macro avg']['recall'],
        'Avg F1-score (macro)': report['macro avg']['f1-score'],
        'Avg Precision (weighted)': report['weighted avg']['precision'],
        'Avg Recall (weighted)': report['weighted avg']['recall'],
        'Avg F1-score (weighted)': report['weighted avg']['f1-score']
    })

E = pd.DataFrame(E).set_index('Model', inplace=False)

In [229]:
E

,Accuracy,Avg Precision (macro),Avg Recall (macro),Avg F1-score (macro),Avg Precision (weighted),Avg Recall (weighted),Avg F1-score (weighted)
Model,,,,,,,
Logistic Regression,0.473684,0.488095,0.353535,0.249477,0.501253,0.473684,0.322342
Support Vector Machine,0.456140,0.152047,0.333333,0.208835,0.208064,0.456140,0.285775
Multinomial Naive Bayes,0.464912,0.486726,0.343434,0.229709,0.499379,0.464912,0.304535
Decision Tree,0.745614,0.772110,0.693065,0.705473,0.759809,0.745614,0.727895
Random Forest,0.622807,0.833520,0.526646,0.490434,0.780957,0.622807,0.542286
K-Nearest Neighbors,0.561404,0.570415,0.491942,0.464600,0.568606,0.561404,0.509256
